<a href="https://colab.research.google.com/github/Devica2000/StanfordTech16LLM/blob/main/Tech16_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homeowork Task 1
Summarize some text

## Task 1 Description

My goal is to use the chat completion API to summarize a song I provide as input. To achieve this, I need to convert the song file (.m4a) into text to form a transcript of its lyrics which can then be given as input to the chat completion.

I collected data in 2 ways:

1. I played the song and recorded the audio on my phone. (Don't Give Up On Me by Andy Grammer) [Link](https://storage.googleapis.com/hw1_audio_files/dontgiveuponme.m4a
)
2. I recited the lyrics of the song 'Stargazing' by Myles Smith on my own and recorded my audio (without any background music, instruments etc.) [Link](https://storage.googleapis.com/hw1_audio_files/stargazing.m4a)

I collected data in the second way since I was not able to transcribe the first song file using SpeechRecognition library in Python nor using the Speech-to-Text API in Google Cloud.

Later, I used Whisper to transcribe both my audio files. It was 'almost' accurate for the first audio and perfect for the second.


In [12]:
%pip install openai

In [13]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-anogt4xy
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-anogt4xy
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://p

In [14]:
from google.colab import auth
auth.authenticate_user()

#Copying the files from GCS bucket to the colab
!gsutil cp gs://hw1_audio_files/dontgiveuponme.m4a /content/dontgiveuponme.m4a
!gsutil cp gs://hw1_audio_files/stargazing.m4a /content/stargazing.m4a

Copying gs://hw1_audio_files/dontgiveuponme.m4a...
/ [1 files][  2.6 MiB/  2.6 MiB]                                                
Operation completed over 1 objects/2.6 MiB.                                      
Copying gs://hw1_audio_files/stargazing.m4a...
/ [1 files][576.3 KiB/576.3 KiB]                                                
Operation completed over 1 objects/576.3 KiB.                                    


In [15]:
#Using whisper to convert audio to text i.e., get song lyrics

import whisper
import pprint

def get_lyrics(audio):
  model = whisper.load_model("small")
  result = model.transcribe(f"{audio}")
  lyrics = result["text"]
  return lyrics

In [16]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')

client = OpenAI(api_key=open_ai_key)

def summarize(lyrics):
    """
    Send a text input to the OpenAI GPT-3.5 model and return its summary.

    This function interacts with the OpenAI API, specifically using the GPT-3.5-turbo model. It takes a user's message as input (song lyrics in our case), sends it to the model, and returns the model's text-only response (summary). The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    lyrics (str): A string containing the songs lyrics as input to the AI.

    Returns:
    str: The summary response generated by the GPT-3.5 model.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "YSummarize the text. Understand the text thoroughly, and then give your output."},
            {"role": "user", "content": f"{lyrics}"}
        ]
    )

    summary = response.choices[0].message.content
    return summary


In [17]:
#getting lyrics of audio 1
audio_1_lyrics = get_lyrics("dontgiveuponme.m4a")
print("The lyrics of the song Don't Give Up On Me by Andy Grammers are:\n")
pprint.pp(audio_1_lyrics)

The lyrics of the song Don't Give Up On Me by Andy Grammers are:

(' I will fight, I will fight for you. I always do until my eyes black and '
 "blue. I will stay, I will stay with you. We'll make it to the other side "
 "like lovers do. I'll reach my hands out in the dark and wait for yours to "
 "enter like a wave for you. A wave for you. I'll give it up, I'll not give it "
 "up, give it up, no no yeah. Even when I'm down on my last plan, even when "
 "they say there's nothing left. So don't give up, I'll not give it up, I'm "
 'not giving up, give it up, no no no yeah. Even when nobody else believes, '
 "I'm not going down that easily. So don't give up on me. Now I will hold, "
 "I'll hold onto you. No matter where I go, I won't shake me loose. I'll reach "
 'my hands out in the dark and wait for yours to enter like a wave for you. A '
 "wave for you. I'll reach my hands out in the dark and wait for yours to "
 "enter like a wave for you. I'll reach my hands out in the dark and wait

In [18]:
#getting summary of audio 1
audio_1_summary = summarize(audio_1_lyrics)
print("The summary of the song Don't Give Up On Me by Andy Grammers is:\n")
pprint.pp(audio_1_summary)

The summary of the song Don't Give Up On Me by Andy Grammers is:

('The text is about perseverance, commitment, and love. The speaker expresses '
 'their determination to fight and stay with their loved one through difficult '
 'times. They emphasize not giving up, even when faced with challenges or lack '
 'of support from others. The speaker promises to hold onto their loved one '
 'and wait for them like a guiding light. This message encourages resilience '
 'and unwavering support in relationships.')


In [19]:
#getting lyrics of audio 2
audio_2_lyrics = get_lyrics("stargazing.m4a")
print("The lyrics of the song Stargazing by Myles Smith are:\n")
pprint.pp(audio_2_lyrics)

The lyrics of the song Stargazing by Myles Smith are:

(' Time stood still, just like a photograph. You made me feel like this would '
 'last forever. Looking in your eyes, I see my whole life. They say you know '
 "it when you know it and I know. Oh oh oh. Promise that you'll hold me close. "
 "Don't let me go. Hey. Take my heart. Don't break it. Love me to the bones. "
 'All this time I wasted, you were right there all along. You and I, '
 'stargazing, intertwining souls. We were never ever strangers. You were right '
 "there all along. I lose my mind when I'm around you. How come I am alive? "
 "When I'm without you, I can't help but feel so lost. I wanna give you all "
 "I've got. They say you know it when you know it and I know. Oh oh oh. "
 "Promise that you'll hold me close. Don't let me go. Take my heart. Don't "
 'break it. Love me to the bones. All this time I wasted, you were right there '
 'all along. You and I, stargazing, intertwining souls. We were never '
 'strangers. Y

In [20]:
#getting summary of audio 2
audio_2_summary = summarize(audio_2_lyrics)
print("The summary of the song Stargazing by Myles Smith is:\n")
pprint.pp(audio_2_summary)

The summary of the song Stargazing by Myles Smith is:

("The text is a lyrical expression of someone's deep feelings and connection "
 'with another person. The narrator talks about feeling like time stood still '
 'in the presence of their loved one, describing a strong bond where they see '
 'their whole life in their eyes. They emphasize how they know their love is '
 'genuine and promise to hold on tight to their partner. The lyrics express a '
 'sense of belonging and as if the loved one had always been a significant '
 "part of the narrator's life. The mention of stargazing and intertwining "
 'souls portrays a profound connection between the two individuals. Overall, '
 'the text conveys a message of deep love, longing, and appreciation for the '
 'enduring presence of the loved one in their life.')


## Task 2
Compare two documents for differences

Here, I am comparing wikipedia pages for ChatGPT and Gemini.
The model outputs key differences and similarities between the two documents in a tabular format w.r.t some metrics. Later, it also outputs a brief summary for each input document.

In [46]:
import wikipedia

#get content for ChatGPT from its Wikipedia page
chatgpt = wikipedia.page("ChatGPT - Wikipedia").content
pprint.pp(chatgpt)

('ChatGPT is a chatbot and virtual assistant developed by OpenAI and launched '
 'on November 30, 2022. Based on large language models (LLMs), it enables '
 'users to refine and steer a conversation towards a desired length, format, '
 'style, level of detail, and language. Successive user prompts and replies '
 'are considered at each conversation stage as context.\n'
 'ChatGPT is credited with starting the AI boom, which has led to ongoing '
 'rapid investment in and public attention to the field of artificial '
 'intelligence (AI). By January 2023, it had become what was then the '
 'fastest-growing consumer software application in history, gaining over 100 '
 "million users and contributing to the growth of OpenAI's current valuation "
 "of $86 billion. ChatGPT's release spurred the release of competing products, "
 'including Gemini, Claude, Llama, Ernie, and Grok. Microsoft launched '
 "Copilot, initially based on OpenAI's GPT-4. In June 2024, a partnership "
 'between Apple Inc.

In [47]:
#get content for Gemini from its Wikipedia page
gemini = wikipedia.page("Gemini (Chatbot) - Wikipedia").content
pprint.pp(gemini)

('Gemini, formerly known as Bard, is a generative artificial intelligence '
 'chatbot developed by Google. Based on the large language model (LLM) of the '
 'same name and developed as a direct response to the meteoric rise of '
 "OpenAI's ChatGPT, it was launched in a limited capacity in March 2023 before "
 'expanding to other countries in May. It was previously based on PaLM, and '
 'initially the LaMDA family of large language models.\n'
 'LaMDA had been developed and announced in 2021, but it was not released to '
 "the public out of an abundance of caution. OpenAI's launch of ChatGPT in "
 'November 2022 and its subsequent popularity caught Google executives '
 'off-guard and sent them into a panic, prompting a sweeping response in the '
 'ensuing months. After mobilizing its workforce, the company launched Bard in '
 'February 2023, which took center stage during the 2023 Google I/O keynote in '
 'May and was upgraded to the Gemini LLM in December. Bard and Duet AI were '
 'unif

In [54]:
  prompt = f"""
  ChatGPT:
  {chatgpt}

  Gemini:
  {gemini}
  """

In [55]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  # response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are an AI that understands the documents from a human given as input. Then, you output the differences and similarities between the documents in a table. You display the differences and similarities in a separate tables. At the end, you also output a brief summary of each document."},
    {"role": "user", "content": prompt}
  ]
)

print(response.choices[0].message.content)

**Differences:**

| Aspect                | ChatGPT                                                                                                                                                                                                       | Gemini                                                                                                                                                                              |
|-----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Developer             | Developed by OpenAI.                                                                                                        